In [77]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape
import pprint
import json
import xmltodict
import pandas as pd
import os

In [78]:
def extractData(inp):
    data=json.loads(jsonString)
    data=data['CHART']['RACE']
    horsieData=[]
    for race in data:
        run={}
        run['distance']=float(race['DISTANCE'])
        run['purse']=float(race['PURSE'])
        if(race['COURSE_ID']=='D'):
            run['track']=1
        else:
            run['track']=0
        run['fr1']=float(race['FRACTION_1'])
        run['fr2']=float(race['FRACTION_2'])
        run['fr3']=float(race['FRACTION_3'])
        run['fr4']=float(race['FRACTION_4'])
        run['fr5']=float(race['FRACTION_5'])
        run['frw']=float(race['WIN_TIME'])
        horse={}
        for entry in race['ENTRY']:
            horse['pp_start']=int(entry['START_POSITION'])
            horse['pp_finish']=int(entry['OFFICIAL_FIN'])
            if(int(entry['OFFICIAL_FIN'])<4):
                horse['winloss']=1
            else:
                horse['winloss']=0
            if(int(entry['OFFICIAL_FIN'])==1):
                horse['first']=1
                horse['second']=0
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==2):
                horse['first']=0
                horse['second']=1
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==3):
                horse['first']=0
                horse['second']=0
                horse['third']=1
                horse['loser']=0
            else:
                horse['first']=0
                horse['second']=0
                horse['third']=0
                horse['loser']=1
            horse['age']=int(entry['AGE'])
            horse['speed_rating']=int(entry['SPEED_RATING'])
            horse['horse']=str(entry['AXCISKEY'])
            horse['jockey name']=str(entry['JOCKEY']['FIRST_NAME'])+' '+str(entry['JOCKEY']['LAST_NAME'])
            horse['trainer name']=str(entry['TRAINER']['FIRST_NAME'])+' '+str(entry['TRAINER']['LAST_NAME'])
            horse['odds']=float(entry['DOLLAR_ODDS'])
            call={}
            for poc in entry['POINT_OF_CALL']:
                if(poc['@WHICH']=='1'):
                    call['poc1']=float(poc['LENGTHS'])
    #                 row['poc1p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='2'):
                    call['poc2']=float(poc['LENGTHS'])
    #                 row['poc2p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='3'):
                    call['poc3']=float(poc['LENGTHS'])
    #                 row['poc3p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='4'):
                    call['poc4']=float(poc['LENGTHS'])
    #                 row['poc4p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='5'):
                    call['poc5']=float(poc['LENGTHS'])
    #                 row['poc5p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='FINAL'):
                    call['pocf']=float(poc['LENGTHS'])
    #                 row['pocfp']=int(poc['POSITION'])
            outp={**run, **horse, **call}
            horsieData.append(outp)
    return horsieData

In [79]:
horsieData=[]

for x in os.listdir('resources/races'):
    file='resources/races/'+str(x)
    with open(file, 'r') as f:
        xmlString = f.read()
    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
    horsieData+=extractData(jsonString)

In [80]:
df=pd.DataFrame(horsieData)
df.head()

,age,distance,first,fr1,fr2,fr3,fr4,fr5,frw,horse,...,pocf,pp_finish,pp_start,purse,second,speed_rating,third,track,trainer name,winloss
0,4,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049053050051052058055056,...,4.5,4,6,26000.0,0,107,0,1,Ari Herbertson,0
1,6,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049051050051061055061063,...,2.8,3,2,26000.0,0,110,1,1,Gus Headley,1
2,5,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049052050052052057061059,...,6.3,6,1,26000.0,0,104,0,1,Mark Glatt,0
3,7,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049050050052060060061060,...,5.5,5,7,26000.0,0,105,0,1,Kristin Mulhall,0
4,4,800.0,1,22.8,46.58,71.25,84.5,0.0,97.5,049053050051054056057063,...,0.0,1,5,26000.0,0,114,0,1,Doug O'Neill,1


In [81]:
# file='resources/races/DMR20180718TCH.xml'
# with open(file, 'r') as f:
#     xmlString = f.read()
# jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
# print(jsonString)

In [82]:
# horseLength=8
# furlong=660
# dist1=1320
# dist2=2640
# dist3=3960
# df['speed1']=((dist1-(horseLength*df['poc1'].astype(float)))/df['fr1'].astype(float)).round(2)
# df['speed2']=((dist2-(horseLength*df['poc2'].astype(float)))/df['fr2'].astype(float)).round(2)
# # df['speed3']=((dist3-(horseLength*df['poc3'].astype(float)))/df['fr3'].astype(float)).round(2)
# df['speedf']=(((df['distance'].astype(float)/100*furlong)-(horseLength*df['pocf'].astype(float)))/df['frw'].astype(float)).round(2)
# df.head()

In [83]:
# data=pd.DataFrame(df[['distance','pp_start','speed1','speed2','speed3','speedf','pp_finish']])
data=pd.DataFrame(df[['distance','purse','track','age','jockey name','trainer name','odds','pp_start','winloss']])
data.head()

,distance,purse,track,age,jockey name,trainer name,odds,pp_start,winloss
0,800.0,26000.0,1,4,William Antongeorgi,Ari Herbertson,4.4,6,0
1,800.0,26000.0,1,6,Kyle Frey,Gus Headley,13.9,2,1
2,800.0,26000.0,1,5,Drayden Van Dyke,Mark Glatt,11.1,1,0
3,800.0,26000.0,1,7,Joseph Talamo,Kristin Mulhall,3.9,7,0
4,800.0,26000.0,1,4,Rafael Bejarano,Doug O'Neill,3.1,5,1


In [84]:
# data2=pd.DataFrame(df[['distance','track','age','jockey','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish','first','second','third','loser']])
# data2.head()

In [85]:
# data.to_csv('output/horseData_20180802v1.0.csv', index=False)
# data2.to_csv('output/horseData_20180801v2.csv', index=False)

In [86]:
# data[data['winloss']==1].sort_values(by='odds', ascending=False)

In [87]:
jockey=pd.read_csv('resources/jockies/jockeystats.csv')
jockey=jockey.rename(columns={'jockey_name':'jockey name'})
# jockey.sort_values(by='jockey name')

In [88]:
trainer=pd.read_csv('resources/trainers/trainerstats.csv')
trainer=trainer.rename(columns={'trainer_name':'trainer name'})
# trainer.sort_values(by='trainer name')

In [89]:
hj=pd.merge(data,jockey, how='left', left_on='jockey name', right_on='jockey name')
hj.sort_values(by='jockey name').head()

,distance,purse,track,age,jockey name,trainer name,odds,pp_start,winloss,j_starts,j_first,j_second,j_third,j_total_win_amt,j_win_perc,j_top_3,j_top_3_perc
789,700.0,150000.0,1,3,Abel Cedillo,Robert Hess,33.2,2,0,1,0,0,0,6000,0.00,0,0.00
397,800.0,75000.0,0,7,Agapito Delgadillo,Victor Garcia,35.5,9,0,9,1,1,2,34975,0.11,4,0.44
394,500.0,30000.0,1,2,Agapito Delgadillo,Richard Baltas,4.9,4,1,9,1,1,2,34975,0.11,4,0.44
602,850.0,34000.0,0,4,Agapito Delgadillo,Victor Garcia,20.9,1,0,9,1,1,2,34975,0.11,4,0.44
24,600.0,30000.0,1,4,Agapito Delgadillo,William Spawr,14.3,1,1,9,1,1,2,34975,0.11,4,0.44


In [90]:
# hj['jockey name'][hj['jockey name'].isnull()]

In [91]:
hjt=pd.merge(hj,trainer, how='left', left_on='trainer name', right_on='trainer name')
hjt.sort_values(by='trainer name').head()

,distance,purse,track,age,jockey name,trainer name,odds,pp_start,winloss,j_starts,...,j_win_perc,j_top_3,j_top_3_perc,t_starts,t_first,t_second,t_third,t_total_win_amt,t_win_perc,t_top_3
21,600.0,30000.0,1,5,Stewart Elliott,Adam Kitchingman,6.4,3,1,39,...,0.05,11,0.28,5,0,1,0,11490,0.0,0.20
232,500.0,60000.0,1,2,Martin Pedroza,Adam Kitchingman,9.8,4,0,32,...,0.00,7,0.22,5,0,1,0,11490,0.0,0.20
274,650.0,20000.0,1,6,Asa Espinoza,Adam Kitchingman,2.5,1,0,63,...,0.14,20,0.32,5,0,1,0,11490,0.0,0.20
97,850.0,60000.0,0,4,Gary Stevens,Alfredo Marquez,14.0,11,0,31,...,0.06,12,0.39,4,0,0,1,3435,0.0,0.25
697,600.0,20000.0,1,4,Ruben Fuentes,Alfredo Marquez,73.8,4,0,22,...,0.18,7,0.32,4,0,0,1,3435,0.0,0.25


In [92]:
# hjt['trainer name'][hjt['t_starts'].isnull()].sort_values().unique()

In [93]:
# trainer['trainer name'].sort_values().unique()

In [94]:
hjt.sort_values(by='trainer name').head()

,distance,purse,track,age,jockey name,trainer name,odds,pp_start,winloss,j_starts,...,j_win_perc,j_top_3,j_top_3_perc,t_starts,t_first,t_second,t_third,t_total_win_amt,t_win_perc,t_top_3
21,600.0,30000.0,1,5,Stewart Elliott,Adam Kitchingman,6.4,3,1,39,...,0.05,11,0.28,5,0,1,0,11490,0.0,0.20
232,500.0,60000.0,1,2,Martin Pedroza,Adam Kitchingman,9.8,4,0,32,...,0.00,7,0.22,5,0,1,0,11490,0.0,0.20
274,650.0,20000.0,1,6,Asa Espinoza,Adam Kitchingman,2.5,1,0,63,...,0.14,20,0.32,5,0,1,0,11490,0.0,0.20
97,850.0,60000.0,0,4,Gary Stevens,Alfredo Marquez,14.0,11,0,31,...,0.06,12,0.39,4,0,0,1,3435,0.0,0.25
697,600.0,20000.0,1,4,Ruben Fuentes,Alfredo Marquez,73.8,4,0,22,...,0.18,7,0.32,4,0,0,1,3435,0.0,0.25


In [95]:
list(hjt.columns.values)

['distance',
 'purse',
 'track',
 'age',
 'jockey name',
 'trainer name',
 'odds',
 'pp_start',
 'winloss',
 'j_starts',
 'j_first',
 'j_second',
 'j_third',
 'j_total_win_amt',
 'j_win_perc',
 'j_top_3',
 'j_top_3_perc',
 't_starts',
 't_first',
 't_second',
 't_third',
 't_total_win_amt',
 't_win_perc',
 't_top_3']

In [97]:
cols=['distance',
 'purse',
 'track',
 'age',
 'odds',
 'pp_start',
 'j_starts',
 'j_first',
 'j_second',
 'j_third',
 'j_total_win_amt',
 'j_win_perc',
 'j_top_3',
 'j_top_3_perc',
 't_starts',
 't_first',
 't_second',
 't_third',
 't_total_win_amt',
 't_win_perc',
 't_top_3',
 'winloss']
outp_df=hjt[cols]
outp_df.head()

,distance,purse,track,age,odds,pp_start,j_starts,j_first,j_second,j_third,...,j_top_3,j_top_3_perc,t_starts,t_first,t_second,t_third,t_total_win_amt,t_win_perc,t_top_3,winloss
0,800.0,26000.0,1,4,4.4,6,5,0,1,0,...,1,0.20,7,0,0,0,3630,0.00,0.00,0
1,800.0,26000.0,1,6,13.9,2,15,0,1,3,...,4,0.27,3,0,1,1,7465,0.00,0.67,1
2,800.0,26000.0,1,5,11.1,1,42,10,3,5,...,18,0.43,24,2,4,4,179100,0.08,0.42,0
3,800.0,26000.0,1,7,3.9,7,45,2,5,3,...,10,0.22,5,0,2,1,23065,0.00,0.60,0
4,800.0,26000.0,1,4,3.1,5,51,4,7,7,...,18,0.35,51,9,9,5,435875,0.18,0.45,1


In [98]:
outp_df.to_csv('output/horseData_20180804v1.1.csv', index=False)